In [ ]:
# !pip install langchain
# !pip install -U langchain-openai
# !pip install -U langchain_neo4j
# !pip install langchain_community
# !pip install neo4j
# !pip install shapely
# !pip install -U pyproj
# !pip install tqdm
# !pip install langchain-community
# !pip install -U pandas

# !pip install -U langchain-neo4j

In [47]:
# import json
# from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import os

from langchain_neo4j import Neo4jGraph
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict


import psycopg2
from neo4j import GraphDatabase
from decimal import Decimal
from shapely import wkb
from pyproj import CRS, Transformer


In [3]:

load_dotenv()

keys = ["NEO4J_USER", "NEO4J_PASSWORD","NEO4J_USER_LOCAL", "NEO4J_PASSWORD_LOCAL"]
for key in keys:
    if key not in os.environ:
        raise Exception(f"Key '{key}' not found not in .env")
    
print("Credentials from .env file successfully loaded")

NEO4J_LOCAL_DATABASE = "busoppdse203"

Credentials from .env file successfully loaded


In [6]:
graph = Neo4jGraph(url="bolt://localhost:7687",    username='neo4j',password='admin123',database=NEO4J_LOCAL_DATABASE )
graph.query("MATCH (n) RETURN n LIMIT 1;")


[{'n': {'name': 'Midtown', 'id': 60}}]

In [7]:
# Initialize the LLM
model_ver = "gpt-4o"
model_provider = 'openai'
model = init_chat_model(model_ver, model_provider=model_provider)


In [8]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[dict]
    answer: str


In [9]:
# Retrieve context 
# def retrieve(state: State):
#     context = graph.query("CALL db.schema.visualization()")
#     return {"context": context}




def retrieve(state: State):
    schema_map = graph.query("CALL apoc.meta.schema()")
    
    context = schema_map 
    return {"context": context}

In [10]:
# Create a prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)



In [11]:
# Generate the answer based on the question and context
def generate(state: State):
    messages = prompt.invoke({"question": state["question"], "context": state["context"]})
    response = model.invoke(messages)
    return {"answer": response.content}







In [12]:
# Define application steps
workflow = StateGraph(State).add_sequence([retrieve, generate])
workflow.add_edge(START, "retrieve")
app = workflow.compile()


In [15]:
# entity_type = 'BlockGroup'
# question = "How is the graph structured?"



question = "What questions can you answer"
# question = f"What attributes are available on a {entity_type} node"
# question = "Are any cities contained in cities"

# question = 'What is the context'

response = app.invoke({"question": question})
print("Answer:", response["answer"])

Answer: Based on the context provided, I can answer the following types of questions:

1. **Node Count Questions**:
   - How many nodes exist for each type (e.g., Business, BlockGroup, City, etc.)?

2. **Relationship Count Questions**:
   - How many specific relationships exist (e.g., how many "adjacent_to" or "contained_in" relationships are there)?

3. **Node Property Questions**:
   - What properties are available for each node type (e.g., properties of Business, BlockGroup, etc.) and their data types?

4. **Relationships Details**:
   - What are the types and directions of relationships between node types (e.g., which nodes have "contained_in" or "adjacent_to" relationships)?

5. **Node Uniqueness and Indexing**:
   - Are properties of nodes unique or indexed?

If you have questions within these areas, I can provide answers based on the context given.


# Text 2 Cypher

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langchain_neo4j import Neo4jGraph

# Initialize the LLM
model = init_chat_model(
    "gpt-4o", 
    model_provider="openai"
)


cypher_model = init_chat_model(
    "gpt-4o", 
    model_provider="openai",
    temperature=0.0
)


# Connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI_LOCAL"),
    username=os.getenv("NEO4J_USER_LOCAL"), 
    password=os.getenv("NEO4J_PASSWORD_LOCAL"),
    database=NEO4J_LOCAL_DATABASE
)

# Create the Cypher QA chain

# Invoke the chain

In [ ]:
from langchain_neo4j import GraphCypherQAChain

cypher_qa = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=model, 
    cypher_llm=cypher_model,
    allow_dangerous_requests=True,
    verbose=True,
)


# # Create the Cypher QA chain
# cypher_qa = GraphCypherQAChain.from_llm(
#     graph=graph, 
#     llm=model, 
#     allow_dangerous_requests=True,
#     verbose=True, 
# )



In [52]:

# Invoke the chain
question = "Find a starbucks with a high rating, look for its location, create a boundaing box of +/- 0.1 degrees and return 5 business locations in there"

response = cypher_qa.invoke({"query": question})
print(response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:Business {name: "Starbucks"})<-[:belongs_to]-(bl:BusinessLocation)
WHERE bl.avg_rating > 4.0
WITH bl
LIMIT 1
WITH bl.latitude AS lat, bl.longitude AS lon
MATCH (bl2:BusinessLocation)
WHERE bl2.latitude > lat - 0.1 AND bl2.latitude < lat + 0.1
  AND bl2.longitude > lon - 0.1 AND bl2.longitude < lon + 0.1
RETURN bl2
LIMIT 5

Full Context:
[{'bl2': {'zip': 92101, 'franchise': 'FRANCHISE', 'address': "Wetzel's Pretzels, 869 W. Harbor Drive, #C2-F, San Diego, CA 92101", 'city': 'San Diego', 'latitude': 32.7085652, 'confidence': 0.9500000000000001, 'reasoning': "Wetzel's Pretzels is a well-known chain specializing in pretzels, indicating it operates as a franchise.", 'geom': '0101000020E6100000DD0F2ED3E54A5DC0B68AB443B25A4040', 'blockgroup': 54023.0, 'url': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x80d95357f0384b23:0x27bdaeb687f1103e?authuser=-1&hl=en&gl=us', 'avg_rating': 3.9, 'name': "Wetzel's Pretze

In [ ]:

# Invoke the chain
question = "What kind of business locations have high ratings in heavily tot populated blockgroups where starbucks" \
" have high ratings? " \
"Find 5 of those businesses that do well but in a blockgroup with few starbucks"
response = cypher_qa.invoke({"query": question})
print(response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:Business)-[:belongs_to]-(bl:BusinessLocation)-[:contained_in]->(bg:BlockGroup)
WHERE bl.avg_rating > 4.0 AND bg.totpop_cy > 1000
WITH bl, bg
MATCH (s:Business {name: "Starbucks"})-[:belongs_to]-(sbl:BusinessLocation)-[:contained_in]->(bg)
WHERE sbl.avg_rating > 4.0
WITH bl, bg, COUNT(sbl) AS starbucks_count
WHERE starbucks_count < 5
RETURN bl.name, bl.categories, bl.avg_rating, bg.totpop_cy
ORDER BY bl.avg_rating DESC
LIMIT 5

Full Context:
[{'bl.name': 'Ocean Reef Inc', 'bl.categories': ['Sportwear manufacturer', 'Entertainment and recreation', 'Importer', 'Sporting goods store'], 'bl.avg_rating': 5.0, 'bg.totpop_cy': 1090}, {'bl.name': 'Chiltonic Hillcrest', 'bl.categories': ['Spa', 'Health spa', 'Medical spa', 'Spa and health club', 'Wellness center'], 'bl.avg_rating': 5.0, 'bg.totpop_cy': 1450}, {'bl.name': 'Crossfit Trifecta', 'bl.categories': ['Gym'], 'bl.avg_rating': 5.0, 'bg.totpop_cy': 1090}, {'bl.n

In [36]:
print(graph.schema)

Node properties:
State {name: STRING, code: STRING, id: INTEGER}
County {name: STRING, id: INTEGER}
City {name: STRING, id: INTEGER}
Community {name: STRING, id: INTEGER}
Zipcode {id: INTEGER, zipcode: INTEGER}
Business {name: STRING, id: INTEGER, categories: LIST, num_locations: INTEGER}
BusinessLocation {name: STRING, id: INTEGER, categories: LIST, zip: INTEGER, franchise: STRING, address: STRING, city: STRING, latitude: FLOAT, confidence: FLOAT, reasoning: STRING, geom: STRING, blockgroup: FLOAT, url: STRING, avg_rating: FLOAT, longitude: FLOAT, location: POINT}
BlockGroup {geom: STRING, x1001fy_x: FLOAT, n01_bus: FLOAT, n14_bus: FLOAT, n37_sales: FLOAT, s23_emp: FLOAT, countyfp: INTEGER, di150_cy: INTEGER, blkgrpce: INTEGER, indmanu_cy: INTEGER, tractce: INTEGER, x1024_x: FLOAT, s22_bus: FLOAT, male25: INTEGER, totpop_cy: INTEGER, n35_bus: FLOAT, fem25: INTEGER, source_cou: STRING, aggregatio: STRING, s16_sales: FLOAT, male30: INTEGER, avghinc_cy: INTEGER, male35: INTEGER, fem30: I